**Задание**

Данные берем отызывы за лето

На вебинаре мы говорили, что долгое время CNN и RNN архитектуры были конурируещими выяснить какая архитектура больше подходит для нашей задачи

1. построить свёрточные архитектуры
2. построить различные архитектуры с RNN
3. построить совместные архитектуры CNN -> RNN или (RNN -> CNN)

In [1]:
# ! pip install stop_words
# ! pip install pymorphy2

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

from keras.losses import SparseCategoricalCrossentropy
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, MaxPool1D, SimpleRNN, LSTM, GRU, Masking, Flatten
from keras.callbacks import TensorBoard 
from keras.callbacks import EarlyStopping 
from tensorflow.keras.utils import to_categorical

In [3]:
! pip install --upgrade xlrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
#Загрузка данных
df = pd.read_excel('/content/отзывы за лето.xls')
df.head(5)

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [5]:
#Разбиваем на трейн и тест
df_train, df_test = train_test_split(df, test_size=0.33, random_state=42)
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_train.shape, df_test.shape

((13841, 3), (6818, 3))

In [6]:
#Предобрабатываем данные
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['Content'] = df_train['Content'].apply(preprocess_text)
df_test['Content'] = df_test['Content'].apply(preprocess_text)


In [7]:
text_corpus_train = df_train['Content'].values
text_corpus_test = df_test['Content'].values

In [8]:
# Определяем токенайзер
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
# Обучаем токенайзер на тренировочном наборе данных - > получаем готовый словарь
tokenizer.fit_on_texts(text_corpus_train)

In [9]:
# На основе полученного словаря токены переводятся в их id и сформировываем последовательности sequences

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

In [10]:
# всего слов в словаре
word_count = len(tokenizer.index_word) + 1

# находим максимальную длину будущей последовательности в тренировочном наборе отзывов
training_length = max([len(i.split()) for i in text_corpus_train])

word_count, training_length

(10293, 113)

In [11]:
X_train = pad_sequences(sequences_train, maxlen=training_length)
X_test = pad_sequences(sequences_test, maxlen=training_length)

X_train[5]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0, 127,   1], dtype=int32)

In [12]:
# Преобразование значений y
le = LabelEncoder()
train_enc_labels = le.fit_transform(df_train['Rating']) 
test_enc_labels = le.transform(df_test['Rating'])
le.classes_

array([1, 2, 3, 4, 5])

In [13]:
num_classes = df_train['Rating'].nunique()
y_train = to_categorical(train_enc_labels, num_classes)
y_test = to_categorical(test_enc_labels, num_classes)

In [14]:
y_train.shape, y_train[3]

((13841, 5), array([0., 0., 0., 0., 1.], dtype=float32))

**CNN (Сверточные нейронные сети)**

In [15]:
model = Sequential()
model.add(Embedding(input_dim=word_count, output_dim=256, input_length=training_length)) 
                    #inputdim -- размер словаря, outputdim -- длина вектора, input_length -- длина входной последовательности
                    #на вход: (батч, input_lenght), на выходе: (батч, input_lenght, output_dim)

model.add(Conv1D(128, 3, padding = 'same'))
                   #64 -- количество 1D-фильтров, длина фильтра -- 3
                   #на выходе (количество объектов, 128 - длина нового эмбединга)

model.add(MaxPool1D(2))  
model.add(Activation("relu"))


model.add(Conv1D(256, 3, padding = 'same'))
model.add(MaxPool1D(2))
model.add(Activation("relu"))
                    #применяем функцию активации к выходу предыдущего слоя

model.add(GlobalMaxPool1D())
                    # в каждой свертке для каждого элемента эмбединга оставляем максимум - получаем один 128-м эмбеддинг

model.add(Dense(128))
               #128-количество выходов
model.add(Activation("relu"))
model.add(Dense(64))
model.add(Activation("relu"))

model.add(Dense(num_classes))
                #num_classes = 5 -- количество выходов
model.add(Activation('softmax'))
                #преобразуем вектор в раcgределение вероятностей

In [16]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])

In [17]:
early_stopping=EarlyStopping(monitor='val_loss', patience = 1, mode = 'min')  


history = model.fit(X_train, y_train,
                    batch_size=256,
                    epochs=10,
                    verbose=1,
                    validation_data = (X_test, y_test),
                    callbacks=[early_stopping]
                    )

Epoch 1/10
55/55 [==============================] - 5s 34ms/step - loss: 0.9450 - accuracy: 0.7124 - val_loss: 0.6892 - val_accuracy: 0.7600
Epoch 2/10
55/55 [==============================] - 1s 24ms/step - loss: 0.6417 - accuracy: 0.7719 - val_loss: 0.6550 - val_accuracy: 0.7702
Epoch 3/10
55/55 [==============================] - 1s 26ms/step - loss: 0.5482 - accuracy: 0.8015 - val_loss: 0.6757 - val_accuracy: 0.7646


In [18]:
cnn_score = model.evaluate(X_test, y_test, batch_size=256, verbose=1)
print('\n')
print('Test score:', cnn_score[0])
print('Test accuracy:', cnn_score[1])

27/27 [==============================] - 0s 9ms/step - loss: 0.6757 - accuracy: 0.7646


Test score: 0.6756554841995239
Test accuracy: 0.7645937204360962


**RNN (Рекуррентные нейронные сети)**

Simple RNN

In [19]:
model = Sequential()

model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(64))  #64 -- количество ячеек, размер выходного пространства - количество скрытых состояний на момент последнего эл-тв послед-сти
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [20]:
model.compile(
    optimizer='adam', 
    loss='binary_crossentropy', 
    metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_data = (X_test, y_test),
                    callbacks = [early_stopping]
                    )

Epoch 1/10
28/28 [==============================] - 13s 385ms/step - loss: 0.4311 - accuracy: 0.6914 - val_loss: 0.3054 - val_accuracy: 0.7124
Epoch 2/10
28/28 [==============================] - 5s 195ms/step - loss: 0.3167 - accuracy: 0.7085 - val_loss: 0.2641 - val_accuracy: 0.7347
Epoch 3/10
28/28 [==============================] - 5s 194ms/step - loss: 0.2753 - accuracy: 0.7334 - val_loss: 0.2442 - val_accuracy: 0.7486
Epoch 4/10
28/28 [==============================] - 5s 190ms/step - loss: 0.2508 - accuracy: 0.7569 - val_loss: 0.2318 - val_accuracy: 0.7570
Epoch 5/10
28/28 [==============================] - 5s 174ms/step - loss: 0.2319 - accuracy: 0.7716 - val_loss: 0.2268 - val_accuracy: 0.7614
Epoch 6/10
28/28 [==============================] - 5s 181ms/step - loss: 0.2137 - accuracy: 0.7852 - val_loss: 0.2362 - val_accuracy: 0.7598


In [21]:
srnn_score = model.evaluate(X_test, y_test, batch_size=512, verbose=1)
print('\n')
print('Test score:', srnn_score[0])
print('Test accuracy:', srnn_score[1])

14/14 [==============================] - 0s 16ms/step - loss: 0.2362 - accuracy: 0.7598


Test score: 0.23622490465641022
Test accuracy: 0.7597535848617554


LSTM (Long short term memory)

In [22]:
model = Sequential()

model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(LSTM(64))  #64 -- количество ячеек, размер выходного пространства - количество скрытых состояний на момент последнего эл-тв послед-сти
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))


model.compile(
    optimizer='adam', 
    loss='binary_crossentropy', 
    metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_data = (X_test, y_test),
                    callbacks = [early_stopping]
                    )

Epoch 1/10
28/28 [==============================] - 7s 70ms/step - loss: 0.5637 - accuracy: 0.4719 - val_loss: 0.3211 - val_accuracy: 0.7125
Epoch 2/10
28/28 [==============================] - 1s 19ms/step - loss: 0.3689 - accuracy: 0.6910 - val_loss: 0.3167 - val_accuracy: 0.7125
Epoch 3/10
28/28 [==============================] - 1s 18ms/step - loss: 0.3417 - accuracy: 0.7007 - val_loss: 0.2849 - val_accuracy: 0.7125
Epoch 4/10
28/28 [==============================] - 1s 19ms/step - loss: 0.2903 - accuracy: 0.7171 - val_loss: 0.2485 - val_accuracy: 0.7379
Epoch 5/10
28/28 [==============================] - 1s 18ms/step - loss: 0.2604 - accuracy: 0.7377 - val_loss: 0.2334 - val_accuracy: 0.7548
Epoch 6/10
28/28 [==============================] - 1s 18ms/step - loss: 0.2420 - accuracy: 0.7576 - val_loss: 0.2274 - val_accuracy: 0.7605
Epoch 7/10
28/28 [==============================] - 1s 18ms/step - loss: 0.2303 - accuracy: 0.7663 - val_loss: 0.2309 - val_accuracy: 0.7622


In [23]:
lstm_score = model.evaluate(X_test, y_test, batch_size=512, verbose=1)
print('\n')
print('Test score:', lstm_score[0])
print('Test accuracy:', lstm_score[1])

14/14 [==============================] - 0s 10ms/step - loss: 0.2309 - accuracy: 0.7622


Test score: 0.23087015748023987
Test accuracy: 0.7622469663619995


GRU (Gated recurrent unit)

In [24]:
model = Sequential()

model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(LSTM(64))  #64 -- количество ячеек, размер выходного пространства - количество скрытых состояний на момент последнего эл-тв послед-сти
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))


model.compile(
    optimizer='adam', 
    loss='binary_crossentropy', 
    metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_data = (X_test, y_test),
                    callbacks = [early_stopping]
                    )

Epoch 1/10
28/28 [==============================] - 6s 68ms/step - loss: 0.5351 - accuracy: 0.5193 - val_loss: 0.3196 - val_accuracy: 0.7125
Epoch 2/10
28/28 [==============================] - 1s 18ms/step - loss: 0.3591 - accuracy: 0.6990 - val_loss: 0.3162 - val_accuracy: 0.7125
Epoch 3/10
28/28 [==============================] - 1s 18ms/step - loss: 0.3271 - accuracy: 0.7035 - val_loss: 0.2717 - val_accuracy: 0.7259
Epoch 4/10
28/28 [==============================] - 1s 21ms/step - loss: 0.2776 - accuracy: 0.7282 - val_loss: 0.2404 - val_accuracy: 0.7504
Epoch 5/10
28/28 [==============================] - 1s 19ms/step - loss: 0.2507 - accuracy: 0.7518 - val_loss: 0.2330 - val_accuracy: 0.7559
Epoch 6/10
28/28 [==============================] - 1s 19ms/step - loss: 0.2349 - accuracy: 0.7646 - val_loss: 0.2285 - val_accuracy: 0.7602
Epoch 7/10
28/28 [==============================] - 1s 18ms/step - loss: 0.2237 - accuracy: 0.7749 - val_loss: 0.2276 - val_accuracy: 0.7644
Epoch 8/10
28

In [25]:
gru_score = model.evaluate(X_test, y_test, batch_size=512, verbose=1)
print('\n')
print('Test score:', gru_score[0])
print('Test accuracy:', gru_score[1])

14/14 [==============================] - 0s 10ms/step - loss: 0.2286 - accuracy: 0.7694


Test score: 0.22855816781520844
Test accuracy: 0.769433856010437


RNN + CNN

In [26]:
# CNN - > RNN

model = Sequential()

model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(LSTM(64,recurrent_dropout=0.2)) 
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

model.compile(
    optimizer='adam', 
    loss='binary_crossentropy', 
    metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_data = (X_test, y_test),
                    callbacks = [early_stopping]
                    )

Epoch 1/10
28/28 [==============================] - 16s 498ms/step - loss: 0.5620 - accuracy: 0.3797 - val_loss: 0.3733 - val_accuracy: 0.7125
Epoch 2/10
28/28 [==============================] - 12s 427ms/step - loss: 0.3816 - accuracy: 0.6731 - val_loss: 0.2762 - val_accuracy: 0.7385
Epoch 3/10
28/28 [==============================] - 12s 426ms/step - loss: 0.3127 - accuracy: 0.7183 - val_loss: 0.2430 - val_accuracy: 0.7520
Epoch 4/10
28/28 [==============================] - 12s 423ms/step - loss: 0.2740 - accuracy: 0.7457 - val_loss: 0.2283 - val_accuracy: 0.7620
Epoch 5/10
28/28 [==============================] - 12s 422ms/step - loss: 0.2479 - accuracy: 0.7657 - val_loss: 0.2262 - val_accuracy: 0.7666
Epoch 6/10
28/28 [==============================] - 12s 428ms/step - loss: 0.2311 - accuracy: 0.7782 - val_loss: 0.2294 - val_accuracy: 0.7686


In [27]:
cnn_rnn_score = model.evaluate(X_test, y_test, batch_size=512, verbose=1)
print('\n')
print('Test score:', cnn_rnn_score[0])
print('Test accuracy:', cnn_rnn_score[1])

14/14 [==============================] - 1s 37ms/step - loss: 0.2294 - accuracy: 0.7686


Test score: 0.22943797707557678
Test accuracy: 0.7685538530349731


In [28]:
# RNN - > CNN
model = Sequential()

model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(LSTM(64,recurrent_dropout=0.2, return_sequences=True))
  #return_sequences = True с- возвращаем последовательность скрытых состояний для каждого токена
model.add(Conv1D(64, 3, padding = 'same', activation = 'relu'))
model.add(GlobalMaxPool1D())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))



model.compile(
    optimizer='adam', 
    loss='binary_crossentropy', 
    metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_data = (X_test, y_test),
                    callbacks = [early_stopping]
                    )

Epoch 1/10
28/28 [==============================] - 17s 530ms/step - loss: 0.5544 - accuracy: 0.6385 - val_loss: 0.3430 - val_accuracy: 0.7125
Epoch 2/10
28/28 [==============================] - 14s 493ms/step - loss: 0.4050 - accuracy: 0.6742 - val_loss: 0.3293 - val_accuracy: 0.7125
Epoch 3/10
28/28 [==============================] - 14s 486ms/step - loss: 0.3849 - accuracy: 0.6870 - val_loss: 0.3206 - val_accuracy: 0.7125
Epoch 4/10
28/28 [==============================] - 14s 487ms/step - loss: 0.3756 - accuracy: 0.6937 - val_loss: 0.3112 - val_accuracy: 0.7125
Epoch 5/10
28/28 [==============================] - 14s 501ms/step - loss: 0.3494 - accuracy: 0.6991 - val_loss: 0.2645 - val_accuracy: 0.7229
Epoch 6/10
28/28 [==============================] - 14s 496ms/step - loss: 0.2938 - accuracy: 0.7261 - val_loss: 0.2384 - val_accuracy: 0.7441
Epoch 7/10
28/28 [==============================] - 14s 487ms/step - loss: 0.2695 - accuracy: 0.7403 - val_loss: 0.2307 - val_accuracy: 0.7624

In [29]:
rnn_cnn_score = model.evaluate(X_test, y_test, batch_size=512, verbose=1)
print('\n')
print('Test score:', rnn_cnn_score[0])
print('Test accuracy:', rnn_cnn_score[1])

14/14 [==============================] - 1s 41ms/step - loss: 0.2224 - accuracy: 0.7727


Test score: 0.2224355787038803
Test accuracy: 0.7726606130599976


Оценка результатов

In [30]:
result = pd.DataFrame({'model': ['CNN', 'SimpleRNN', 'LSTM', 'GRU', 'CNN->LSTM', 'LSTM->CNN'], 
                       'accuracy': [cnn_score[1], srnn_score[1], lstm_score[1], gru_score[1], cnn_rnn_score[1], rnn_cnn_score[1]]})
result.sort_values(by='accuracy', ascending=False)

,model,accuracy
5,LSTM->CNN,0.772661
3,GRU,0.769434
4,CNN->LSTM,0.768554
0,CNN,0.764594
2,LSTM,0.762247
1,SimpleRNN,0.759754


Наилучший результат показала совместная модель LSTM>CNN, однако в целом все модели достаточно неплохо справились с задачей. 